In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
io = 'data_cleaning.xlsx'
sheetname = 'Usage'
df = pd.read_excel(io, sheet_name = 'Usage', header = None)
df.columns = ['usage']
df['usage'] = df['usage'].str.lower().str.strip()
df.head()

,usage
0,3 pm mon 24th-mar-2014___0.384 kwh
1,5am 15-aug-2014___1.201 kwh
2,__8pm thu 20-mar-2014____1.523 kwh
3,6pm 23rd-apr-2014___0.424 kwh
4,_1am friday 19th-dec-2014___0.209 kwh


In [3]:
pattern = r'_{1}'

df['usage'] = df['usage'].str.replace(pattern, ' ')

df.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_6424\1908811310.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['usage'] = df['usage'].str.replace(pattern, ' ')


,usage
0,3 pm mon 24th-mar-2014 0.384 kwh
1,5am 15-aug-2014 1.201 kwh
2,8pm thu 20-mar-2014 1.523 kwh
3,6pm 23rd-apr-2014 0.424 kwh
4,1am friday 19th-dec-2014 0.209 kwh


In [4]:
pattern = r'(\d+)\s*([ap]m)'

df[['hour', 'am_pm']] = df['usage'].str.extract(pattern)

df['hour'] = pd.to_numeric(df['hour'])

df.head()

,usage,hour,am_pm
0,3 pm mon 24th-mar-2014 0.384 kwh,3,pm
1,5am 15-aug-2014 1.201 kwh,5,am
2,8pm thu 20-mar-2014 1.523 kwh,8,pm
3,6pm 23rd-apr-2014 0.424 kwh,6,pm
4,1am friday 19th-dec-2014 0.209 kwh,1,am


In [5]:
pattern = r'(\d{1,2})(?:st|nd|rd|th)?-(\w+)-(\d{4})'

df[['day', 'month', 'year']] = df['usage'].str.extract(pattern)

df['day'] = pd.to_numeric(df['day'])
df['year'] = pd.to_numeric(df['year'])

df.head()

,usage,hour,am_pm,day,month,year
0,3 pm mon 24th-mar-2014 0.384 kwh,3,pm,24,mar,2014
1,5am 15-aug-2014 1.201 kwh,5,am,15,aug,2014
2,8pm thu 20-mar-2014 1.523 kwh,8,pm,20,mar,2014
3,6pm 23rd-apr-2014 0.424 kwh,6,pm,23,apr,2014
4,1am friday 19th-dec-2014 0.209 kwh,1,am,19,dec,2014


In [6]:
pattern = r'(\d+\.\d+)\s*kwh'

df['kwh'] = df['usage'].str.extract(pattern)

df['kwh'] = pd.to_numeric(df['kwh'])

df.head()

,usage,hour,am_pm,day,month,year,kwh
0,3 pm mon 24th-mar-2014 0.384 kwh,3,pm,24,mar,2014,0.384
1,5am 15-aug-2014 1.201 kwh,5,am,15,aug,2014,1.201
2,8pm thu 20-mar-2014 1.523 kwh,8,pm,20,mar,2014,1.523
3,6pm 23rd-apr-2014 0.424 kwh,6,pm,23,apr,2014,0.424
4,1am friday 19th-dec-2014 0.209 kwh,1,am,19,dec,2014,0.209


In [7]:
from datetime import datetime

df['hour'] = df.apply(lambda x: x['hour'] + 12 if x['am_pm'] == 'pm' else x['hour'], axis=1)

df['month'] = df['month'].apply(lambda x: datetime.strptime(x, '%b').month)

df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

df.head()

,usage,hour,am_pm,day,month,year,kwh,datetime
0,3 pm mon 24th-mar-2014 0.384 kwh,15,pm,24,3,2014,0.384,2014-03-24 15:00:00
1,5am 15-aug-2014 1.201 kwh,5,am,15,8,2014,1.201,2014-08-15 05:00:00
2,8pm thu 20-mar-2014 1.523 kwh,20,pm,20,3,2014,1.523,2014-03-20 20:00:00
3,6pm 23rd-apr-2014 0.424 kwh,18,pm,23,4,2014,0.424,2014-04-23 18:00:00
4,1am friday 19th-dec-2014 0.209 kwh,1,am,19,12,2014,0.209,2014-12-19 01:00:00


In [8]:
month_mapping = {
    1 : 'jan' , 2 : 'feb', 3 : 'mar', 4 : 'apr', 5 : 'may', 6 : 'jun',
    7 : 'jul', 8 : 'aug', 9 : 'sep', 10 : 'oct', 11 : 'nov', 12 : 'dec'
}

df['month'] = df['month'].map(month_mapping)

df.head()

,usage,hour,am_pm,day,month,year,kwh,datetime
0,3 pm mon 24th-mar-2014 0.384 kwh,15,pm,24,mar,2014,0.384,2014-03-24 15:00:00
1,5am 15-aug-2014 1.201 kwh,5,am,15,aug,2014,1.201,2014-08-15 05:00:00
2,8pm thu 20-mar-2014 1.523 kwh,20,pm,20,mar,2014,1.523,2014-03-20 20:00:00
3,6pm 23rd-apr-2014 0.424 kwh,18,pm,23,apr,2014,0.424,2014-04-23 18:00:00
4,1am friday 19th-dec-2014 0.209 kwh,1,am,19,dec,2014,0.209,2014-12-19 01:00:00


In [9]:
df['h24'] = df['datetime'].dt.hour
df.head()

,usage,hour,am_pm,day,month,year,kwh,datetime,h24
0,3 pm mon 24th-mar-2014 0.384 kwh,15,pm,24,mar,2014,0.384,2014-03-24 15:00:00,15
1,5am 15-aug-2014 1.201 kwh,5,am,15,aug,2014,1.201,2014-08-15 05:00:00,5
2,8pm thu 20-mar-2014 1.523 kwh,20,pm,20,mar,2014,1.523,2014-03-20 20:00:00,20
3,6pm 23rd-apr-2014 0.424 kwh,18,pm,23,apr,2014,0.424,2014-04-23 18:00:00,18
4,1am friday 19th-dec-2014 0.209 kwh,1,am,19,dec,2014,0.209,2014-12-19 01:00:00,1


In [10]:
day_of_week = {
    
    0: 'Monday',
    1: 'Tuesday',
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
    
}

In [11]:
df['d_o_w'] = df['datetime'].dt.dayofweek.map(day_of_week)
df.head()

,usage,hour,am_pm,day,month,year,kwh,datetime,h24,d_o_w
0,3 pm mon 24th-mar-2014 0.384 kwh,15,pm,24,mar,2014,0.384,2014-03-24 15:00:00,15,Monday
1,5am 15-aug-2014 1.201 kwh,5,am,15,aug,2014,1.201,2014-08-15 05:00:00,5,Friday
2,8pm thu 20-mar-2014 1.523 kwh,20,pm,20,mar,2014,1.523,2014-03-20 20:00:00,20,Thursday
3,6pm 23rd-apr-2014 0.424 kwh,18,pm,23,apr,2014,0.424,2014-04-23 18:00:00,18,Wednesday
4,1am friday 19th-dec-2014 0.209 kwh,1,am,19,dec,2014,0.209,2014-12-19 01:00:00,1,Friday


In [12]:
df.drop('usage', inplace = True, axis = 1)
df.head()

,hour,am_pm,day,month,year,kwh,datetime,h24,d_o_w
0,15,pm,24,mar,2014,0.384,2014-03-24 15:00:00,15,Monday
1,5,am,15,aug,2014,1.201,2014-08-15 05:00:00,5,Friday
2,20,pm,20,mar,2014,1.523,2014-03-20 20:00:00,20,Thursday
3,18,pm,23,apr,2014,0.424,2014-04-23 18:00:00,18,Wednesday
4,1,am,19,dec,2014,0.209,2014-12-19 01:00:00,1,Friday


In [13]:
df.sort_values(by = ['datetime'], ascending = True, inplace = True)
df.head()

,hour,am_pm,day,month,year,kwh,datetime,h24,d_o_w
6316,1,am,1,jan,2014,0.340,2014-01-01 01:00:00,1,Wednesday
5068,2,am,1,jan,2014,0.332,2014-01-01 02:00:00,2,Wednesday
4106,3,am,1,jan,2014,0.396,2014-01-01 03:00:00,3,Wednesday
3239,4,am,1,jan,2014,0.388,2014-01-01 04:00:00,4,Wednesday
1815,5,am,1,jan,2014,2.354,2014-01-01 05:00:00,5,Wednesday


In [14]:
df = df.reset_index(drop = True)
df.head()

,hour,am_pm,day,month,year,kwh,datetime,h24,d_o_w
0,1,am,1,jan,2014,0.340,2014-01-01 01:00:00,1,Wednesday
1,2,am,1,jan,2014,0.332,2014-01-01 02:00:00,2,Wednesday
2,3,am,1,jan,2014,0.396,2014-01-01 03:00:00,3,Wednesday
3,4,am,1,jan,2014,0.388,2014-01-01 04:00:00,4,Wednesday
4,5,am,1,jan,2014,2.354,2014-01-01 05:00:00,5,Wednesday


# QUESTION 1 

In [15]:
round(df['kwh'].mean(),3)

0.782

#### Answer : b

# Question 2

In [16]:
round(df[df['month'] == 'feb'].kwh.mean(),3)

0.834

#### Answer : d

# Question 3

In [17]:
q3 = pd.DataFrame(round(df.groupby('d_o_w').kwh.mean(),3))
q3.sort_values(by = 'kwh' , ascending = False)

,kwh
d_o_w,
Sunday,0.898
Saturday,0.887
Friday,0.753
Thursday,0.747
Wednesday,0.733
Monday,0.729
Tuesday,0.726


#### Answer : a

# Question 4

In [18]:
pd.DataFrame(df['kwh'].rolling(4).sum().sort_values(ascending = False)).max()

kwh    17.237
dtype: float64

#### Answer : a

# Question 5

In [19]:
monthly_flex = {
    
    'jan' : 0.20,
    'feb' : 0.19,
    'mar' : 0.17,
    'apr' : 0.18,
    'may' : 0.22,
    'jun' : 0.27,
    'jul' : 0.24,
    'aug' : 0.19,
    'sep' : 0.18,
    'oct' : 0.15,
    'nov' : 0.14,
    'dec' : 0.19
    
}

In [20]:
monthly_kwh = df.groupby(by = ['month']).agg({'kwh' : 'sum'}).reset_index()
monthly_kwh

,month,kwh
0,apr,151.666
1,aug,635.819
2,dec,568.948
3,feb,559.436
4,jan,1042.660
5,jul,1255.351
6,jun,899.904
7,mar,422.655
8,may,416.813
9,nov,232.420


In [21]:
monthly_price = pd.DataFrame(monthly_kwh['month'].map(monthly_flex) * monthly_kwh['kwh'])
monthly_price

,0
0,27.29988
1,120.80561
2,108.10012
3,106.29284
4,208.53200
5,301.28424
6,242.97408
7,71.85135
8,91.69886
9,32.53880


In [22]:
round(monthly_price.sum(),3)

0    1421.206
dtype: float64

#### Answer : b

# Question 6

In [261]:
Monthly_annual_cost = 1421.206

In [271]:
round(df['kwh'].sum() * 0.21,3)

1438.104

In [272]:
No_Flex_Rate_annual_cost = 1438.104 

In [273]:
df.head()

,hour,am_pm,day,month,year,kwh,datetime,h24,d_o_w
0,1,am,1,jan,2014,0.340,2014-01-01 01:00:00,1,Wednesday
1,2,am,1,jan,2014,0.332,2014-01-01 02:00:00,2,Wednesday
2,3,am,1,jan,2014,0.396,2014-01-01 03:00:00,3,Wednesday
3,4,am,1,jan,2014,0.388,2014-01-01 04:00:00,4,Wednesday
4,5,am,1,jan,2014,2.354,2014-01-01 05:00:00,5,Wednesday


In [289]:
df['hour_interval'] = df['hour'].astype(str) + ' am to ' + (df['hour'] + 1).astype(str) + ' am'

hourly_kwh_sum = df.groupby('hour_interval')['kwh'].sum()

df = df.merge(hourly_kwh_sum, on='hour_interval', suffixes=('', '_sum'))

In [292]:
df

,hour,am_pm,day,month,year,kwh,datetime,h24,d_o_w,hour_interval,kwh_sum
0,1,am,1,jan,2014,0.340,2014-01-01 01:00:00,1,Wednesday,1 am to 2 am,93.966
1,1,am,2,jan,2014,0.472,2014-01-02 01:00:00,1,Thursday,1 am to 2 am,93.966
2,1,am,3,jan,2014,0.448,2014-01-03 01:00:00,1,Friday,1 am to 2 am,93.966
3,1,am,4,jan,2014,0.612,2014-01-04 01:00:00,1,Saturday,1 am to 2 am,93.966
4,1,am,5,jan,2014,0.540,2014-01-05 01:00:00,1,Sunday,1 am to 2 am,93.966
...,...,...,...,...,...,...,...,...,...,...,...
8755,24,pm,27,dec,2014,0.706,2014-12-28 00:00:00,0,Sunday,24 am to 25 am,212.546
8756,24,pm,28,dec,2014,0.581,2014-12-29 00:00:00,0,Monday,24 am to 25 am,212.546
8757,24,pm,29,dec,2014,0.567,2014-12-30 00:00:00,0,Tuesday,24 am to 25 am,212.546
8758,24,pm,30,dec,2014,0.644,2014-12-31 00:00:00,0,Wednesday,24 am to 25 am,212.546


# ???????????????????